In [108]:
import urllib.request
import requests
import pandas as pd
pd.set_option('max_colwidth', 800)

import functools

## Request Prostitution-related Data from the WPRDC API

In [23]:
# Base URL suggested by WPRDC
# url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=e03a89dd-134a-4ee8-a2bd-62c40aeebc6f&limit=5&q=title:jones'  

In [65]:
test_url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=e03a89dd-134a-4ee8-a2bd-62c40aeebc6f&limit=20&q={"OFFENSES":"5902"}'  

In [70]:
r = requests.get(test_url).json()
r['success']

True

In [71]:
r

{'help': 'https://data.wprdc.org/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 20,
  'q': {'OFFENSES': '5902'},
  'records_format': 'objects',
  'resource_id': 'e03a89dd-134a-4ee8-a2bd-62c40aeebc6f',
  'total_estimation_threshold': None,
  'records': [{'_id': 73054,
    'PK': '2047953',
    'CCR': '22063258',
    'AGE': 39,
    'GENDER': 'M',
    'RACE': 'B',
    'ARRESTTIME': '2022-05-18T11:30:00',
    'ARRESTLOCATION': 'Zone 1',
    'OFFENSES': '3304 Criminal Mischief. / 5902(a)(1) Prostitution  / 3124.1 Sexual Assault / 5902(b) Promoting Prostitution / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain  / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from / 3121(a)(3) Rape of unconscious person / 3123(a)(3) IDSI with unconscious person / 3126(a)(4) Indecent assault of un

## Format arrest data

In [72]:
data = pd.DataFrame(r['result']['records'])
data = data.set_index('_id')
data.index.rename('id', inplace=True)
data

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES
id,,,,,,,,,,,,,,,,,
73054,2047953,22063258,39,M,B,2022-05-18T11:30:00,Zone 1,3304 Criminal Mischief. / 5902(a)(1) Prostitution / 3124.1 Sexual Assault / 5902(b) Promoting Prostitution / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from / 3121(a)(3) Rape of unconscious person / 3123(a)(3) IDSI with unconscious person / 3126(a)(4) Indecent assault of unconscious person / 3131(c)(2) Unlawful dissemination of intimate image -- Person depicted is NOT a minor (M2),Zone 1,None,1,None,None,None,NaN,NaN,0.083129
64121,2040340,21141377,36,M,B,2021-08-05T13:15:00,"500 Block Smithfield ST Pittsburgh, PA 15222",5902(b) Promoting Prostitution / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(6) Transporting for the purpose of engaging in prostitution / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from,"500 Block Smithfield ST Pittsburgh, PA 15222",Central Business District,2,201,6,6,-79.998119,40.440340,0.083129
57618,2034399,21016098,40,M,B,2021-01-28T13:25:00,"200 Block Chauvet DR Pittsburgh, PA 15275",907 Possessing Instruments of Crime. / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(6) Transporting for the purpose of engaging in prostitution / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from,"200 Block Chauvet DR Pittsburgh, PA 15275",Outside City,OSC,5599,None,None,-80.184332,40.450256,0.081707
48965,2027109,20092091,21,M,W,2020-05-29T09:30:00,"900 Block Second AV Pittsburgh, PA 16219",5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(6) Transporting for the purpose of engaging in prostitution / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from,"200 Block Coltart AV Pittsburgh, PA 15213",Central Oakland,4,405,3,3,-79.959854,40.438730,0.081707
76160,2050776,22128453,36,M,B,2022-08-11T16:00:00,"200 Block Technology Center DR Pittsburgh, PA 15213",5902(b) Promoting Prostitution / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(6) Transporting for the purpose of engaging in prostitution / 5902(b)(8) Prostitution - receiving benefit from,"200 Block Technology Center DR Pittsburgh, PA 15213",South Oakland,4,409,None,None,NaN,NaN,0.079652
74243,2048980,22088525,39,M,B,2022-06-14T10:45:00,Zone 2,3121 Rape. / 5902(b)(1) Promoting Prostitution - Keeping Business or House / 5902(b)(3) Promoting Prostitution - Causing another to become or remain / 5902(b)(7) Permitting place to be regularly used for prostitution / 5902(b)(8) Prostitution - receiving benefit from / 3126(a)(4) Indecent assault of unconscious person,Zone 1,None,1,None,None,None,NaN,NaN,0.079652
32428,2012926,19075179,35,F,B,2019-04-22T18:49:00,"Howard ST & E North AV Pittsburgh, AR 15212",903 Criminal Conspiracy. / 5902(a)(1) Prostitution / 9501 Bench Warrant / 5902(a)(2) Prostitution - Loitering for purpose of / 5902(b)(5) Procuring a prostitute for another,"Middle ST & Foreland ST Pittsburgh, PA 15212",East Allegheny,1,2304,1,1,-79.999393,40.454577,0.076422
32805,2013262,19080308,27,M,W,2019-04-29T15:15:00,"100 Block E Werterfront DR Pittsburgh, PA 15120",903 Criminal Conspiracy. / 5902(b) Promoting Prostitution / 5902(b)(6) Transporting for the purpose of engaging in prostitution /

## Tease out types of prostitution-related charges

In [122]:
charges_dict = {
    "providers" : {
        "5902\s?\(a\)": 'Prostitution',
        "5902\s?\(a\)\(1\)": 'Prostitution',
        "5902\s?\(a\)\(2\)": 'Prostitution - Loitering for',
    },
    "third-party" : {
        "5902\s?\(b\)": 'Promoting',
        "5902\s?\(b\)\(1\)": 'Promoting - Keeping house or business for ',
        "5902\s?\(b\)\(2\)": 'Promoting - Finding brothelworker',
        "5902\s?\(b\)\(3\)": 'Promoting - Encouraging',
        "5902\s?\(b\)\(4\)": 'Promoting - Soliciting patron for',
        "5902\s?\(b\)\(5\)": 'Promoting - Procuring prostitute for patron',
        "5902\s?\(b\)\(6\)": 'Promoting - Transporting for',
        "5902\s?\(b\)\(7\)": 'Promoting - Leasing or regular use of place',
        "5902\s?\(b\)\(8\)": 'Promoting - Receiving benefit',
        "5902\s?\(d\)": 'Living off prostitutes',
    },
    "clients": {   
        "5902\s?\(e\)": 'Patronizing prostitutes',
    }
}

charges_by_p = {
    "providers" : ["5902(a)", "5902(a)(1)", "5902(a)(2)"],
    "third-party" : ["5902(b)(1)", "5902(b)(2)", "5902(b)(3)", "5902(b)(4)", "5902(b)(5)", "5902(b)(6)", "5902(b)(7)", "5902(b)(8)", "5902(d)"], 
    "clients" : ["5902(e)"]
}

In [123]:
## Providers
provider_masks = []
for key in charges_dict["providers"]: 
    provider_masks.append(data.OFFENSES.str.contains(key))

data.OFFENSES.str.contains('|'.join(charges_dict["providers"]))

id
73054     True
64121    False
57618    False
48965    False
76160    False
74243    False
32428     True
32805    False
54046     True
23392    False
76161    False
32806    False
35966    False
22452    False
46495     True
32429     True
36816     True
20424     True
20408     True
67812     True
Name: OFFENSES, dtype: bool

In [124]:
data.OFFENSES.str.contains('|'.join(charges_dict["third-party"]))

id
73054     True
64121     True
57618     True
48965     True
76160     True
74243     True
32428     True
32805     True
54046     True
23392     True
76161     True
32806     True
35966     True
22452     True
46495     True
32429     True
36816     True
20424    False
20408    False
67812    False
Name: OFFENSES, dtype: bool

In [22]:
possible_client <- str_c("5902(e)")
possible_charges <- str_c("5902(a)", "5902(b)", "5902(c)", "5902(d)", "5902 (a)", "5902 (b)", "5902 (c)", "5902 (d)", sep = 

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y,rank OFFENSES
id,,,,,,,,,,,,,,,,,
73054,2047953,22063258,39,M,B,2022-05-18T11:30:00,Zone 1,3304 Criminal Mischief. / 5902(a)(1) Prostitut...,Zone 1,None,1,None,None,None,NaN,NaN,0.083129
64121,2040340,21141377,36,M,B,2021-08-05T13:15:00,"500 Block Smithfield ST Pittsburgh, PA 15222",5902(b) Promoting Prostitution / 5902(b)(1) Pr...,"500 Block Smithfield ST Pittsburgh, PA 15222",Central Business District,2,201,6,6,-79.998119,40.440340,0.083129
57618,2034399,21016098,40,M,B,2021-01-28T13:25:00,"200 Block Chauvet DR Pittsburgh, PA 15275",907 Possessing Instruments of Crime. / 5902(b)...,"200 Block Chauvet DR Pittsburgh, PA 15275",Outside City,OSC,5599,None,None,-80.184332,40.450256,0.081707
48965,2027109,20092091,21,M,W,2020-05-29T09:30:00,"900 Block Second AV Pittsburgh, PA 16219",5902(b)(1) Promoting Prostitution - Keeping Bu...,"200 Block Coltart AV Pittsburgh, PA 15213",Central Oakland,4,405,3,3,-79.959854,40.438730,0.081707
76160,2050776,22128453,36,M,B,2022-08-11T16:00:00,"200 Block Technology Center DR Pittsburgh, PA ...",5902(b) Promoting Prostitution / 5902(b)(3) Pr...,"200 Block Technology Center DR Pittsburgh, PA ...",South Oakland,4,409,None,None,NaN,NaN,0.079652
74243,2048980,22088525,39,M,B,2022-06-14T10:45:00,Zone 2,3121 Rape. / 5902(b)(1) Promoting Prostitution...,Zone 1,None,1,None,None,None,NaN,NaN,0.079652
32428,2012926,19075179,35,F,B,2019-04-22T18:49:00,"Howard ST & E North AV Pittsburgh, AR 15212",903 Criminal Conspiracy. / 5902(a)(1) Prostitu...,"Middle ST & Foreland ST Pittsburgh, PA 15212",East Allegheny,1,2304,1,1,-79.999393,40.454577,0.076422
32805,2013262,19080308,27,M,W,2019-04-29T15:15:00,"100 Block E Werterfront DR Pittsburgh, PA 15120",903 Criminal Conspiracy. / 5902(b) Promoting P...,"100 Block E Werterfront DR Pittsburgh, PA 15120",Outside City,OSC,5599,None,None,NaN,NaN,0.076422
54046,2031349,20183214,40,M,B,2020-09-30T19:40:00,"100 Block Washington PL Pittsburgh, PA 15219",903 Criminal Conspiracy. / 907 Possessing Inst...,"100 Block Washington PL Pittsburgh, PA 15219",Central Business District,2,201,None,None,-80.061512,40.344060,0.076422
